In [ ]:
library(dplyr)
library(monocle)
library(RColorBrewer)

## Monocle requires an object of the CellDataSet class
http://cole-trapnell-lab.github.io/monocle-release/docs/

1. The class is compatible with previous Bioconductor class
2. three input files are required:
    1. a numeric matrix of expression values, where rows are genes, and columns are cells
    2. phenoData, where rows are cells, and columns are cell attributes
    3. featureData, where rows are features (eg, genes) and columns are gene attributes.
3. Do NOT normalize data yourself!!
4. UMIs or read counts are better modeled with negative binomial.

## 1. read in data for monocle

In [ ]:
var <- read.csv('/oasis/tscc/scratch/CSHL_single_cell_2019/for_monocle/var2318_mncl.csv', header = T, row.names = 1)
obs <- read.csv('/oasis/tscc/scratch/CSHL_single_cell_2019/for_monocle/obs2318_mncl.csv', header = T, row.names = 1)
dim(var)
dim(obs)

In [ ]:
exp <- read.table('/oasis/tscc/scratch/CSHL_single_cell_2019/for_monocle/exp2318_mncl.txt')
dim(exp)

In [ ]:
# create a CellDataSet object

pd <- AnnotatedDataFrame(data = obs)
fd <- AnnotatedDataFrame(data = var)
colnames(exp) <- rownames(pd)
rownames(exp) <- rownames(fd)
head(exp)

In [ ]:
# create the CellDataSet object

cds <- newCellDataSet(cellData=as.matrix(exp), phenoData=pd, featureData=fd, expressionFamily=negbinomial.size())

In [ ]:
# take a look of the data structure of CellDataSet object
str(cds)

In [ ]:
slotNames(cds)

## 2. pre-processing
Since we have already filtered the dataset in scanpy, we will go directly to normalization

In [ ]:
dim(cds)

In [ ]:
#Normalize the count data

cds <- estimateSizeFactors(cds)

In [ ]:
#Calculate dispersions to filter for highly variable genes

cds <- estimateDispersions(cds)

In [ ]:
head(fData(cds))

In [ ]:
cds <- detectGenes(cds, min_expr = 0.1)
dim(fData(cds))

In [ ]:
head(fData(cds))

In [ ]:
#Filter for Stem, EP, TA, and Enterocytes, since our main goal here is to reconstruct the differentiation trajectory of the enterocytes
# generate a cell_mask 

cell_types = as.character(pData(cds)$louvain_final)             
cell_mask = rep(FALSE, length(cell_types))                      
cells_to_keep = c("Stem", "EP", "TA", "Enterocyte")             
for (item in cells_to_keep) {cell_mask = cell_mask | startsWith(cell_types, item)}
print("Number of cells after filtering:")
print(sum(cell_mask))
print("")

In [ ]:
# apply cell_mask

cds <- cds[,cell_mask]
dim(cds)

In [ ]:
str(fData(cds))

### 3. pseudotiming projection

In [ ]:
#Do dimensionality reduction
# DDRTree: discriminative dimensionality reduction with Trees

cds <- reduceDimension(cds, norm_method = 'vstExprs', reduction_method='DDRTree', verbose = F, max_components = 7)

In [ ]:
#Run for the first time to get the ordering

cds <- orderCells(cds)

### Excercise 1: Could you find out what changes did you make to the cds object by ordering the cells?
your code here:

In [ ]:
# 'root cell' need to be manually defined.
# Find the correct root state that corresponds to the 'Stem' cluster

tab1 <- table(pData(cds)$State, pData(cds)$louvain_final)
id = which(colnames(tab1) == 'Stem')
root_name = names(which.max(tab1[,id]))

In [ ]:
#Run a second time to get the correct root state that overlaps with Stem cells

cds <- orderCells(cds, root_state=root_name)

### Have fun plotting!!

In [ ]:
#Visualize pseudotime
options(repr.plot.width=12, repr.plot.height=8)  # set the size of the plot

plot_cell_trajectory(cds, color_by="Pseudotime")
# the dark circles mark the transitions 

In [ ]:
plot_cell_trajectory(cds, color_by="louvain_final", show_branch_points = FALSE, cell_size = 1.0)

In [ ]:
options(repr.plot.width=16, repr.plot.height=10)

plot_cell_trajectory(cds, color_by='State', cell_size = 1.0) + facet_wrap(~louvain_final, nrow=2)

### Excercise 2: Could you plot the pseudotime trajectory with other dimensions?
Hits:
1. when we did dimension reduction, we generated 7 components. In the above plot, we are plotting component 1 and 2.
2. help(plot_cell_trajectory)

your code here:


### Excercise 3: The data came from two different mice (specified as 'donor') in two independent experiments.
Can you check whether there is 'batch effect'?

your code here:


In [ ]:
#Get a nice colour map
custom_colour_map = brewer.pal(length(unique(pData(cds)$louvain_final)),'Paired')

#Find the correct root state that coresponds to the 'Stem' cluster
tab1 <- table(pData(cds)$State, pData(cds)$louvain_final)
id = which(colnames(tab1) == 'Stem')
root_name = names(which.max(tab1[,id]))

When cells are colored and plotted based on cell states, it gives us an intuition that everytime there is a branch point, a new cell state is called.
Technically, this is achieved through a depth-first search (DFS) of the learned principal tree, starting from the root cell. Everytime a branch point was reached, a new state was assigned.

You may also notice the branch point indexes are not in order. The index for the branch point is only assigned during the trajectory plotting process. So the 'cell state' and 'branching point' were determined by two different algorithms, thus not matching each other, unfortuantely.

https://github.com/cole-trapnell-lab/monocle-release/issues/167

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

plot_complex_cell_trajectory(cds, color_by = 'State', show_branch_points = T, 
                             cell_size = 2, cell_link_size = 1, root_states = c(root_name)) +
scale_size(range = c(0.2, 0.2)) +
theme(legend.position="left", legend.title=element_blank(), legend.text=element_text(size=rel(1))) +
guides(colour = guide_legend(override.aes = list(size=3))) 


In [ ]:
# Visualize with our cluster labels

options(repr.plot.width=8, repr.plot.height=6)

plot_complex_cell_trajectory(cds, color_by = 'louvain_final', show_branch_points = T, 
                             cell_size = 2, cell_link_size = 1, root_states = c(root_name)) +
scale_size(range = c(0.2, 0.2)) +
theme(legend.position="left", legend.title=element_blank(), legend.text=element_text(size=rel(1.5))) +
guides(colour = guide_legend(override.aes = list(size=6))) + 
scale_color_manual(values = custom_colour_map)

In [ ]:
# Visualize with our cluster labels
options(repr.plot.width=10, repr.plot.height=10)

plot_complex_cell_trajectory(cds, color_by = 'louvain_final', show_branch_points = T, 
                             cell_size = 2, cell_link_size = 1, root_states = c(root_name)) +
scale_size(range = c(0.2, 0.2)) +
theme(legend.position="left", legend.title=element_blank(), legend.text=element_text(size=rel(1.5))) +
guides(colour = guide_legend(override.aes = list(size=6))) + 
scale_color_manual(values = custom_colour_map) + 
facet_wrap(~louvain_final, nrow=2)

#### plot genes of interests along the pseudotime trajectory

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

my_genes <- c('Lgr5','Ascl2','Apoa4','Neurod1')
gene_mask <- row.names(fData(cds)) %in% my_genes

cds_subset <- cds[gene_mask,]
plot_genes_in_pseudotime(cds_subset, color_by = "louvain_final")

## 4. Finding genes that change as a function of pseudotime

Monocle's main job is to put cells in order of progress through a biological process (such as cell differentiation) without knowing which genes to look at ahead of time.

Monocle assigns each cell a "pseudotime" value, which records its progress through the process in the experiment. The model can test against changes as a function of this value. Monocle uses the VGAM package to model a gene's expression level as a smooth, nonlinear function of pseudotime. 

In [ ]:
diff_test_res <- differentialGeneTest(cds, fullModelFormulaStr = '~sm.ns(Pseudotime)')

In [ ]:
# select top 6 genes to plot

diff_test_res_ord <- diff_test_res[order(diff_test_res$qval),]
my_pseudotime_gene <- row.names(diff_test_res_ord)[1:5]
my_pseudotime_gene

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

plot_genes_in_pseudotime(cds[my_pseudotime_gene,], color_by = "louvain_final")

### clustering genes by pseudotemporal expression pattern
Monocle can group genes that have similar trends, so you can analyze these groups to see what they have in common. 
plot_pseudotime_heatmap function generate a hierarchical clustering for the input genes. Clusters are for the genes.

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

pseudotime_gene4cluster <- row.names(diff_test_res_ord)[1:50]

pseudotime_cluster <- plot_pseudotime_heatmap(cds[pseudotime_gene4cluster,], num_clusters = 6, 
                        cores = 2,show_rownames = T, return_heatmap = T)

In [ ]:
# we can check how many genes in each cluster: these are the genes covariate along the pseudotime trajectory.

gene_cluster <- cutree(pseudotime_cluster$tree_row, 6)
table(gene_cluster)

In [ ]:
# we can pull out the genes in each cluster. 
names(gene_cluster[gene_cluster == 1])

In [ ]:
str(names(gene_cluster[gene_cluster == 1]))

### 5. Analyzing branches in single-cell trajectory
What genes change as cells pass from the early developmental stage the top left of the tree through the branch? What genes are differentially expressed between the branches?

BEAM: branched expression analysis modeling

In [ ]:
options(repr.plot.width=5, repr.plot.height=4)
plot_cell_trajectory(cds, color_by = 'louvain_final', cell_size = 0.5)

In [ ]:
options(repr.plot.width=5, repr.plot.height=4)
plot_cell_trajectory(cds, x = 1, y = 3, color_by = 'louvain_final', cell_size = 0.5)

In [ ]:
beam_res <- BEAM(cds, branch_point = 2, core = 2)

In [ ]:
beam_res <- beam_res[order(beam_res$qval),]
head(beam_res)

This heatmap shows changes in both lineages. It also requires that you choose a branch point to inspect. Columns are points in pseudotime, rows are genes, and the beginning of pseudotime is in the middle of the heatmap.
1. cell fate 1 matches up with mature/immature enterocytes
2. cell fate 2 matches up with EP cells.

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
my_branched_heatmap <- plot_genes_branched_heatmap(cds[row.names(subset(beam_res, qval < 1e-100)),],
                                                   branch_point = 2,
                                                   num_clusters = 8,
                                                   cores = 2,
                                                   show_rownames = TRUE,
                                                   return_heatmap = TRUE)

Let's look at this data!

In [ ]:
head(my_branched_heatmap$annotation_row)

In [ ]:
dim(my_branched_heatmap$annotation_row)

In [ ]:
table(my_branched_heatmap$annotation_row$Cluster)

In [ ]:
my_row <- my_branched_heatmap$annotation_row
my_row <- data.frame(cluster = my_row$Cluster, gene = row.names(my_row), stringsAsFactors = F)
head(my_row)

In [ ]:
write.csv(my_row, '/home/ucsd-train20/ucsd-train20/data/processed_data/degenes_bp2.csv', row.names = T)

Now, we will go to a different branch point 4, to investigate how stem cells are different from EP cell.
plot_genes_branched_pseudotime function shows two kinetic trends, one for each lineage. 

In [ ]:
stemcell_genes <- c('Lgr5', 'Ascl2', 'Slc12a2', 'Axin2', 'Olfm4', 'Gkn3')
stemcell_genes <- row.names(subset(cds, row.names(cds)%in% stemcell_genes))

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

plot_genes_branched_pseudotime(cds[stemcell_genes,],
                       branch_point = 4,
                       color_by = "louvain_final",
                       ncol = 1) +
scale_color_manual(values = custom_colour_map)

### Excercise 4:
We only looked at enterocyte differentiation in this example.
Can you investigate the trajectory from stem cell, TA, to paneth cells?
Can you identify the transcriptomic changes along with this stem->TA->paneth cells trajectory?



In [ ]:
sessionInfo()